# 뉴스 본문 데이터 수집

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [ ]:
# 뉴스 링크 데이터 가져오기
import pandas as pd
df_news = pd.read_csv('daum_economy_news.csv')
df_news.head()

In [ ]:
len(df_news)

In [ ]:
# 첫 번째 뉴스 링크 가져오기
df_news['link'][0]

In [ ]:
# 랭체인 WebBaseLoader로 뉴스 본문 가져오기

import bs4
from langchain_community.document_loaders import WebBaseLoader

url = df_news['link'][0]

loader = WebBaseLoader(
    web_paths=[url, ],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=["article_view"]
        )
    ),
)
docs = loader.load()
len(docs)

In [ ]:
# Document 객체 확인
docs[0]

In [ ]:
# Document 객체의 page_content 속성 확인
docs[0].page_content.strip()

In [ ]:
# 각 링크를 순회하며 뉴스 본문 가져오기
import time

def get_news_content(url):
    loader = WebBaseLoader(
        web_paths=[url],
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=["article_view"]
            )
        ),
    )
    docs = loader.load()
    time.sleep(1)
    return docs[0].page_content.strip()

In [ ]:
df_news['content'] = df_news['link'].apply(get_news_content)
df_news.head()

In [ ]:
# 뉴스 본문 데이터 저장
df_news.to_csv('daum_economy_news_content.csv', index=False)

In [ ]:
# df_news = pd.read_csv('daum_economy_news_content.csv')

# 뉴스 본문에서 언론사 이름, 기자 이름, 이메일 주소 등을 추출 

In [ ]:
# 뉴스 본문 데이터 확인하기
text = df_news['content'].iloc[0]
text

In [ ]:
# 정규표현식 사용 - 기자 이름 추출
import re
name_pattern = r'([가-힣]{2,4} 기자)'
name_matches = re.findall(name_pattern, text)
name = name_matches[0] if name_matches else None
name

In [ ]:
# 정규표현식 사용 - 이메일 주소 추출
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
email_matches = re.findall(email_pattern, text)
email = email_matches[0] if email_matches else None
email

In [ ]:
# LLM 활용 - 기자 이름, 이메일 주소 추출
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# prompt
prompt_template = """다음 뉴스 본문에서 기자 이름, 이메일 주소를 추출합니다.

<뉴스 본문>
{text}
</뉴스 본문>

기자 이름:
이메일 주소:
"""

prompt = PromptTemplate.from_template(prompt_template)

# LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125",
                 api_key=OPENAI_API_KEY)

# output parser
output_parser = StrOutputParser()

# Chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": text})

response

# 뉴스 본문 요약

In [ ]:
# prompt
prompt_template = """다음 뉴스 본문을 3가지 요점으로 요약합니다. 각 요점은 1줄로 작성합니다.

<뉴스 본문>
{text}
</뉴스 본문>

요점 1:
요점 2:
요점 3:
"""

prompt = PromptTemplate.from_template(prompt_template)

# LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125",
                 api_key=OPENAI_API_KEY)

# output parser
output_parser = StrOutputParser()

# Chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": text})

response

# 뉴스 키워드 추출

In [ ]:
# prompt
prompt_template = """다음 뉴스 본문에서 3가지 키워드를 추출합니다. 각 키워드는 쉼표(,)로 구분합니다.

<뉴스 본문>
{text}
</뉴스 본문>

키워드: 
"""

prompt = PromptTemplate.from_template(prompt_template)

# LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125", api_key=OPENAI_API_KEY)

# output parser
output_parser = StrOutputParser()

# Chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": text})

response

## create_extraction_chain 활용하여 요약, 추출

In [ ]:
text

In [ ]:
from langchain.chains import create_extraction_chain
from langchain_openai import ChatOpenAI

# 스키마 정의
schema = {
    "properties": {
        "뉴스_제목": {"type": "string"},
        "뉴스_카테고리": {"type": "string"},
        "뉴스_요약": {"type": "string"},
        "뉴스_키워드": {"type": "string"},
        "기자_이름": {"type": "string"},
        "기자_이메일": {"type": "string"},
    },
    "required": ["뉴스_제목", "뉴스_카테고리", "뉴스_요약", "뉴스_키워드", "기자_이름", "기자_이메일"],
}

# LLM 생성
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125", api_key=OPENAI_API_KEY)

# Chain 생성
chain = create_extraction_chain(schema=schema, llm=llm)

# Chain 실행
response = chain.invoke(text)

# 결과 확인
response

In [ ]:
# prompt 추가

from langchain_core.prompts import ChatPromptTemplate

prompt_template = """Extract and save the relevant entities mentioned \
in the following passage together with their properties.

Only extract the properties mentioned in the 'information_extraction' function.

If a property is not present and is not required in the function parameters, do not include it in the output.

For keywords, extract at most 3 keywords.
For categories, use the following categories: '정치', '경제', '사회', '문화', '세계', '과학', '기술', '스포츠', '엔터', '건강'.

Passage:
{input}
""" 

prompt = ChatPromptTemplate.from_template(prompt_template)

chain = create_extraction_chain(schema=schema, llm=llm, prompt=prompt)

# Chain 실행
response = chain.invoke(text)

# 결과 확인
response